In [1]:
import pandas as pd
import plotly.express as px
import pprint
pp = pprint.PrettyPrinter(indent=4)

In [36]:
current_total_atom_supply = 300000000
current_community_pool_balance = 1140000
current_security_subsidy_rate = 13.5
current_security_subsidy = (current_total_atom_supply * (current_security_subsidy_rate/100))/12
security_subsidy_decay = 10
security_subsidy_termination = 36 
issuance = 10000000
issuance_reduction_epoch_rate = 1
issuance_reduction_amount = 12
steady_state_issuance = 300000
time_until_steady_state_issuance = 36
distrbution_module_pct_to_community_pool = 5
timeline_length = 60

### Form the logic for each Category

In [5]:
# Outflow - Security Subsidy
def calculate_outflow_security_subsidy(month, security_subsidy_termination, current_security_subsidy, security_subsidy_decay, distrbution_module_pct_to_community_pool):
    if month > security_subsidy_termination : 
        return 0
    else: 
        return(current_security_subsidy * pow((1-security_subsidy_decay/100),month)*(1-distrbution_module_pct_to_community_pool/100))

# outflow_security_subsidy = calculate_outflow_security_subsidy(month, security_subsidy_termination, current_security_subsidy, security_subsidy_decay, distrbution_module_pct_to_community_pool)


In [6]:
# Outflow - Distribution to Community Pool
def calculate_outflow_distribution_community_pool(month, security_subsidy_termination, current_security_subsidy, security_subsidy_decay, distrbution_module_pct_to_community_pool):
    if month > security_subsidy_termination : 
        return 0
    else: 
        return(current_security_subsidy * pow((1-security_subsidy_decay/100),month)*(distrbution_module_pct_to_community_pool/100))
    
# outflow_distribution_community_pool = calculate_outflow_distribution_community_pool(month, security_subsidy_termination, current_security_subsidy, security_subsidy_decay, distrbution_module_pct_to_community_pool)


In [7]:
# Inflow - Issued ATOM (new)
def calculate_inflow_issued_atom_new(month, time_until_steady_state_issuance, issuance, issuance_reduction_amount, steady_state_issuance):
    if month >= time_until_steady_state_issuance : 
        return steady_state_issuance
    else: 
        return((issuance-200000) * pow((1-issuance_reduction_amount/100),month)+200000)
    
# inflow_issued_atom_new = calculate_inflow_issued_atom_new(month, time_until_steady_state_issuance, issuance, issuance_reduction_amount, steady_state_issuance)


In [8]:
# Total	- Treasury Pool Balance
    
def calculate_total_treasury_pool_balance(outflow_distribution_community_pool, outflow_security_subsidy, inflow_issued_atom_new):
    total_outflow = outflow_distribution_community_pool + outflow_security_subsidy
    total_treasury_pool_balance = inflow_issued_atom_new - total_outflow
    return total_treasury_pool_balance

# total_treasury_pool_balance = calculate_total_treasury_pool_balance(outflow_distribution_community_pool, outflow_security_subsidy, inflow_issued_atom_new)

In [ ]:
# Total	Community Pool Balance
current_total_comm_pool = current_community_pool_balance
for month in range(timeline_length + 1):
    if month > security_subsidy_termination : 
        distributed_comm_pool = 0
    else: 
        distributed_comm_pool = current_security_subsidy * pow((1-security_subsidy_decay/100),month)*(distrbution_module_pct_to_community_pool/100)
    
    current_total_comm_pool += distributed_comm_pool
    print(current_total_comm_pool)

In [30]:
# Total	- Cumulatively Issued ATOM (new)
total_cumulatively_issued_atom_new = 0
# total_cumulatively_issued_atom_new += calculate_inflow_issued_atom_new

In [ ]:
# Total - Total ATOM Supply
total_atom_supply = current_total_atom_supply

for month in range(timeline_length + 1):
    if month >= time_until_steady_state_issuance : 
        issued_supply = (steady_state_issuance)
    else: 
        issued_supply = (issuance-200000) * pow((1-issuance_reduction_amount/100),month)+200000

    total_atom_supply += issued_supply
    print('\nMonth ', month, ' : ', total_atom_supply)

In [10]:
total_atom_supply = current_total_atom_supply

def calculate_total_atom_supply(total_atom_supply,inflow_issued_atom_new ):
    total_atom_supply += inflow_issued_atom_new
    return total_atom_supply

In [11]:
# Old_Regime 
total_atom_supply_old = current_total_atom_supply
issued_atom_old = 0
cumulatively_issued_atom_old = issued_atom_old

def calculate_old_regime(total_atom_supply_old, issued_atom_old, cumulatively_issued_atom_old, current_security_subsidy_rate):
    total_atom_supply_old += issued_atom_old
    issued_atom_old = total_atom_supply_old*current_security_subsidy_rate/100/12
    cumulatively_issued_atom_old += issued_atom_old
    return (total_atom_supply_old, issued_atom_old, cumulatively_issued_atom_old)


### Combine the logics into one function

In [31]:
total_treasury_pool_balance = 0
current_total_comm_pool = current_community_pool_balance
total_cumulatively_issued_atom_new = 0
total_atom_supply = current_total_atom_supply
total_atom_supply_old = current_total_atom_supply
issued_atom_old = 0
cumulatively_issued_atom_old = issued_atom_old

for month in range(timeline_length + 1):
    outflow_security_subsidy = calculate_outflow_security_subsidy(month, security_subsidy_termination, current_security_subsidy, security_subsidy_decay, distrbution_module_pct_to_community_pool)
    outflow_distribution_community_pool = calculate_outflow_distribution_community_pool(month, security_subsidy_termination, current_security_subsidy, security_subsidy_decay, distrbution_module_pct_to_community_pool)
    inflow_issued_atom_new = calculate_inflow_issued_atom_new(month, time_until_steady_state_issuance, issuance, issuance_reduction_amount, steady_state_issuance)
    total_treasury_pool_balance += calculate_total_treasury_pool_balance(outflow_distribution_community_pool, outflow_security_subsidy, inflow_issued_atom_new)
    current_total_comm_pool += outflow_distribution_community_pool
    total_cumulatively_issued_atom_new += inflow_issued_atom_new
    total_atom_supply = calculate_total_atom_supply(total_atom_supply,inflow_issued_atom_new)
    total_atom_supply_old,issued_atom_old, cumulatively_issued_atom_old = calculate_old_regime(total_atom_supply_old, issued_atom_old, cumulatively_issued_atom_old, current_security_subsidy_rate)

    print(month, outflow_security_subsidy, outflow_distribution_community_pool, inflow_issued_atom_new, total_treasury_pool_balance, current_total_comm_pool, total_cumulatively_issued_atom_new, total_atom_supply, total_atom_supply_old,issued_atom_old, cumulatively_issued_atom_old)

0 3206250.0 168750.0 10000000.0 6625000.0 1308750.0 10000000.0 310000000.0 300000000 3375000.0 3375000.0
1 2885625.0 151875.0 8824000.0 12411500.0 1460625.0 18824000.0 318824000.0 303375000.0 3412968.75 6787968.75
2 2597062.5 136687.5 7789120.0 17466870.0 1597312.5 26613120.0 326613120.0 306787968.75 3451364.6484375 10239333.3984375
3 2337356.2500000005 123018.75000000003 6878425.6 21884920.6 1720331.25 33491545.6 333491545.6 310239333.3984375 3490192.500732422 13729525.899169922
4 2103620.625 110716.875 6077014.528 25747597.628000002 1831048.125 39568560.128 339568560.128 313729525.8991699 3529457.166365661 17258983.065535583
5 1893258.5625000002 99645.18750000001 5371772.78464 29126466.66264 1930693.3125 44940332.91264 344940332.91264004 317258983.0655356 3569163.559487276 20828146.62502286
6 1703932.70625 89680.66875000001 4751160.050483201 32084013.338123202 2020373.98125 49691492.9631232 349691492.96312326 320828146.6250229 3609316.6495315074 24437463.274554364
7 1533539.435625000

### Populate the result into a table

In [37]:
# Initialize lists to store data
data = []
columns = ["Month", "Outflow Security Subsidy", "Outflow Distribution to Community Pool",
           "Inflow Issued ATOM (new)", "Total Treasury Pool Balance",
           "Current Community Pool Balance", "Total Cumulatively Issued ATOM (new)",
           "Total ATOM Supply", "Total ATOM Supply (old)", "Issued ATOM (old)",
           "Cumulatively Issued ATOM (old)"]

# Combined logic
total_treasury_pool_balance = 0
current_total_comm_pool = current_community_pool_balance
total_cumulatively_issued_atom_new = 0
total_atom_supply = current_total_atom_supply
total_atom_supply_old = current_total_atom_supply
issued_atom_old = 0
cumulatively_issued_atom_old = issued_atom_old

for month in range(timeline_length + 1):
    outflow_security_subsidy = calculate_outflow_security_subsidy(month, security_subsidy_termination, current_security_subsidy, security_subsidy_decay, distrbution_module_pct_to_community_pool)
    outflow_distribution_community_pool = calculate_outflow_distribution_community_pool(month, security_subsidy_termination, current_security_subsidy, security_subsidy_decay, distrbution_module_pct_to_community_pool)
    inflow_issued_atom_new = calculate_inflow_issued_atom_new(month, time_until_steady_state_issuance, issuance, issuance_reduction_amount, steady_state_issuance)
    total_treasury_pool_balance += calculate_total_treasury_pool_balance(outflow_distribution_community_pool, outflow_security_subsidy, inflow_issued_atom_new)
    current_total_comm_pool += outflow_distribution_community_pool
    total_cumulatively_issued_atom_new += inflow_issued_atom_new
    total_atom_supply = calculate_total_atom_supply(total_atom_supply,inflow_issued_atom_new)
    total_atom_supply_old,issued_atom_old, cumulatively_issued_atom_old = calculate_old_regime(total_atom_supply_old, issued_atom_old, cumulatively_issued_atom_old, current_security_subsidy_rate)

    # print(month, outflow_security_subsidy, outflow_distribution_community_pool, inflow_issued_atom_new, total_treasury_pool_balance, current_total_comm_pool, total_cumulatively_issued_atom_new, total_atom_supply, total_atom_supply_old,issued_atom_old, cumulatively_issued_atom_old)

    # Append data for the current month to the list
    data.append([month, outflow_security_subsidy, outflow_distribution_community_pool, inflow_issued_atom_new,
                 total_treasury_pool_balance, current_total_comm_pool, total_cumulatively_issued_atom_new,
                 total_atom_supply, total_atom_supply_old, issued_atom_old, cumulatively_issued_atom_old])

# Create a DataFrame from the lists
df = pd.DataFrame(data, columns=columns)

# # Transpose the DataFrame
# df_transposed = df.transpose()

# # Print the transposed DataFrame
# print(df_transposed)


In [38]:
df

,Month,Outflow Security Subsidy,Outflow Distribution to Community Pool,Inflow Issued ATOM (new),Total Treasury Pool Balance,Current Community Pool Balance,Total Cumulatively Issued ATOM (new),Total ATOM Supply,Total ATOM Supply (old),Issued ATOM (old),Cumulatively Issued ATOM (old)
0,0,3206250.000,168750.000,1.000000e+07,6.625000e+06,1.308750e+06,1.000000e+07,3.100000e+08,3.000000e+08,3.375000e+06,3.375000e+06
1,1,2885625.000,151875.000,8.824000e+06,1.241150e+07,1.460625e+06,1.882400e+07,3.188240e+08,3.033750e+08,3.412969e+06,6.787969e+06
2,2,2597062.500,136687.500,7.789120e+06,1.746687e+07,1.597312e+06,2.661312e+07,3.266131e+08,3.067880e+08,3.451365e+06,1.023933e+07
3,3,2337356.250,123018.750,6.878426e+06,2.188492e+07,1.720331e+06,3.349155e+07,3.334915e+08,3.102393e+08,3.490193e+06,1.372953e+07
4,4,2103620.625,110716.875,6.077015e+06,2.574760e+07,1.831048e+06,3.956856e+07,3.395686e+08,3.137295e+08,3.529457e+06,1.725898e+07
...,...,...,...,...,...,...,...,...,...,...,...
56,56,0.000,0.000,3.000000e+05,6.128171e+07,2.793285e+06,9.434741e+07,3.943474e+08,5.613054e+08,6.314685e+06,2.676201e+08
57,57,0.000,0.000,3.000000e+05,6.158171e+07,2.793285e+06,9.464741e+07,3.946474e+08,5.676201e+08,6.385726e+06,2.740058e+08
58,58,0.000,0.000,3.000000e+05,6.188171e+07,2.793285e+06,9.494741e+07,3.949474e+08,5.740058e+08,6.457565e+06,2.804633e+08
59,59,0.000,0.000,3.000000e+05,6.218171e+07,2.793285e+06,9.524741e+07,3.952474e+08,5.804633e+08,6.530213e+06,2.869936e+08


### Graphs

In [39]:
# Issued ATOM
fig = px.line(df, x='Month', y=['Inflow Issued ATOM (new)', 'Issued ATOM (old)'],
              labels={'value': 'Value', 'variable': 'Category'},
              title='Issued ATOM',
              template='plotly_white')

# Show the plot
fig.show()


In [42]:
# Cumulatively Issued ATOM
fig = px.line(df, x='Month', y=['Total Cumulatively Issued ATOM (new)', 'Cumulatively Issued ATOM (old)'],
              labels={'value': 'Value', 'variable': 'Category'},
              title='Cumulatively Issued ATOM',
              template='plotly_white',
              line_dash_sequence=['solid', 'dot'])

# Show the plot
fig.show()
